In [2]:
# 검색을 받아 첫번째 url주소를 추출
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Hannanum

# from konlpy.tag import Okt
from gensim.models import Word2Vec
import pandas as pd
from urllib.request import Request, urlopen


def get_first_search_result_url(query):
    # 웹드라이버 초기화
    driver = webdriver.Chrome()

    # Google 검색 페이지 열기
    driver.get(f"https://www.google.com/search?q={query}" + " 인재상")

    # 첫 번째 검색 결과 제목의 URL 가져오기
    try:
        first_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h3.LC20lb.MBeuO.DKV0Md"))
        )
        url = first_title.find_element(By.XPATH, "./parent::a").get_attribute("href")
    except:
        url = None

    # 웹드라이버 종료
    driver.quit()
    return url


query = input("검색어를 입력하세요: ")
first_url = get_first_search_result_url(query)


# 명사만 추출하는 함수 정의
def extract_nouns_from_html(first_url):
    # 웹사이트의 HTML 코드 가져오기
    response = requests.get(first_url)

    # 인코딩 설정
    response.encoding = response.apparent_encoding
    html = response.text

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

    # HTML에서 텍스트 추출
    text = soup.get_text()
    # Hannanum 형태소 분석기 초기화
    hannanum = Hannanum()
    # 명사만 추출하여 리스트에 저장
    nouns = hannanum.nouns(text)
    return nouns


# 명사만 추출
nouns = extract_nouns_from_html(first_url)

# # 중복제거 하지 않고 리스트로 출력하여 단어의 범위를 줄이지 않음.
# # 중복 제거를 위해 set으로 변환 후 출력
unique_nouns = set(nouns)
for noun in unique_nouns:
    print(noun)
print("추출된 명사의 갯수: ", len(nouns))


# ==========================관련단어 코드
# Word2Vec 모델 로드
def load_word2vec_model(model_path):
    model = Word2Vec.load(model_path)
    return model


# 인재상에 해당하는 키워드 추출
def extract_related_keywords(nouns, model, target_words, threshold=0.6):
    related_keywords = []
    for word in target_words:
        for noun in nouns:
            try:
                similarity = model.wv.similarity(noun, word)
                if similarity >= threshold:
                    related_keywords.append(noun)
            except:
                pass
    return related_keywords


# Word2Vec 모델 로드
model_path = "wiki.model"
model = load_word2vec_model(model_path)

# 인재상에 해당하는 키워드 리스트
target_words = ["인재", "소통", "능력", "책임", "성실", "열정", "꿈", "사람"]

# 인재상에 해당하는 키워드 추출
related_keywords = extract_related_keywords(nouns, model, target_words)

unique_keywords = set(related_keywords)
print("인재상에 해당하는 키워드:", len(unique_keywords), "개")

df = pd.DataFrame({"인재상에 해당하는 키워드(중복제거)": list(unique_keywords)})
df

The chromedriver version (122.0.6261.128) detected in PATH at c:\Users\kosmo\Desktop\kosmo_baeksn\project3\resumate\data\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.59); currently, chromedriver 123.0.6312.58 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (122.0.6261.128) detected in PATH at c:\Users\kosmo\Desktop\kosmo_baeksn\project3\resumate\data\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.59); currently, chromedriver 123.0.6312.58 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry


KeyboardInterrupt: 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Hannanum
from gensim.models import Word2Vec
import pandas as pd
from selenium.webdriver.chrome.options import Options

# Chrome 옵션 생성
chrome_options = Options()
# 사용자 에이전트 설정
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
)
chrome_options.add_argument("headless")

# WebDriver 초기화
driver = webdriver.Chrome(options=chrome_options)


def get_first_search_result_url(query):
    """
    Google 검색에서 첫 번째 결과의 URL을 반환합니다.
    """
    try:
        # Google 검색 페이지 열기
        driver.get(f"https://www.google.com/search?q={query}" + " 인재상")

        # 첫 번째 검색 결과 제목의 URL 가져오기
        first_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h3.LC20lb.MBeuO.DKV0Md"))
        )
        url = first_title.find_element(By.XPATH, "./parent::a").get_attribute("href")
        return url
    except Exception as e:
        print("Error occurred while getting the first search result URL:", e)
        return None


def extract_nouns_from_html(url):
    """
    주어진 URL의 웹사이트에서 텍스트를 추출하고 명사를 추출하여 반환합니다.
    """
    try:
        # 웹사이트의 HTML 코드 가져오기
        response = requests.get(url)
        response.raise_for_status()  # 오류 발생 시 예외 처리
        response.encoding = response.apparent_encoding
        html = response.text
        print(f"[Debug] extract_nouns_from_html.html\n{html[:100]}")
        # BeautifulSoup을 사용하여 HTML 파싱해주는 구조를 만들어 줌
        soup = BeautifulSoup(html, "html.parser")

        # HTML에서 텍스트 추출
        text = soup.get_text()
        print(f"text\n{html[:100]}")
        # Hannanum 형태소 분석기 초기화
        hannanum = Hannanum()
        # 명사만 추출하여 리스트에 저장
        nouns = hannanum.nouns(text)
        return nouns

    except Exception as e:
        print("Error occurred while extracting nouns from HTML:", e)
        return []


def extract_related_keywords(nouns, model, target_words, threshold=0.6):
    """
    주어진 명사 리스트에서 Word2Vec 모델을 사용하여 목표 단어와의 연관성이 높은 단어를 추출하여 반환합니다.
    """
    related_keywords = []
    for word in target_words:
        for noun in nouns:
            try:
                similarity = model.wv.similarity(noun, word)
                if similarity >= threshold:
                    related_keywords.append(noun)
            except KeyError:
                pass
    return related_keywords


def main():
    query = input("검색어를 입력하세요: ")
    first_url = get_first_search_result_url(query)

    if first_url:
        nouns = extract_nouns_from_html(first_url)
        print("HTML에서 텍스트를 성공적으로 추출하였고, 명사를 추출하였습니다.")
        print("추출된 명사의 개수:", len(nouns))

        model_path = "wiki.model"
        model = Word2Vec.load(model_path)

        target_words = [
            "인재",
            "소통",
            "능력",
            "책임",
            "성실",
            "열정",
            "꿈",
            "사람",
        ]
        related_keywords = extract_related_keywords(nouns, model, target_words)

        print("인재상에 해당하는 키워드:", len(related_keywords), "개")
        df = pd.DataFrame({"인재상에 해당하는 키워드(중복제거)": related_keywords})
        print(df)

    else:
        print("검색 결과 URL을 가져오지 못했습니다.")


if __name__ == "__main__":
    main()

[Debug] extract_nouns_from_html.html
<!doctype html><html data-countrycode="kr" dir="ltr" lang="ko"><head>
<meta http-equiv="Content-Type
text
<!doctype html><html data-countrycode="kr" dir="ltr" lang="ko"><head>
<meta http-equiv="Content-Type


In [6]:
from bs4 import BeautifulSoup
import requests
from konlpy.tag import Hannanum
from gensim.models import Word2Vec
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_first_search_result_url(query):
    """
    Google 검색에서 첫 번째 결과의 URL을 반환합니다.
    """
    try:
        driver = webdriver.Chrome()
        driver.get(f"https://www.google.com/search?q={query}" + " 인재상")
        first_title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h3.LC20lb.MBeuO.DKV0Md"))
        )
        url = first_title.find_element(By.XPATH, "./parent::a").get_attribute("href")
        return url
    except Exception as e:
        print("Error occurred while getting the first search result URL:", e)
        return None
    finally:
        driver.quit()


def extract_nouns(text):
    """
    주어진 텍스트에서 명사를 추출합니다.
    """
    try:
        hannanum = Hannanum()
        nouns = hannanum.nouns(text)
        return nouns
    except Exception as e:
        print("Error occurred while extracting nouns:", e)
        return []


def extract_related_keywords(nouns, model, target_words, threshold=0.6):
    """
    주어진 명사 리스트와 Word2Vec 모델에서 관련 키워드를 추출합니다.
    """
    related_keywords = set()
    for target_word in target_words:
        for noun in nouns:
            try:
                similarity = model.wv.similarity(noun, target_word)
                if similarity >= threshold:
                    related_keywords.add(noun)
            except KeyError:
                pass
    return list(related_keywords)


def main():
    query = input("검색어를 입력하세요: ")
    first_url = get_first_search_result_url(query)

    if first_url:
        try:
            response = requests.get(first_url)
            response.raise_for_status()
            response.encoding = response.apparent_encoding
            html = response.text
            soup = BeautifulSoup(html, "html.parser")
            text = soup.get_text()

            nouns = extract_nouns(text)
            print("HTML에서 텍스트를 성공적으로 추출하였고, 명사를 추출하였습니다.")
            print("추출된 명사의 개수:", len(nouns))

            model_path = "wiki.model"
            model = Word2Vec.load(model_path)

            target_words = [
                "인재",
                "소통",
                "능력",
                "책임",
                "성실",
                "열정",
                "꿈",
                "사람",
            ]
            related_keywords = extract_related_keywords(nouns, model, target_words)

            print("인재상에 해당하는 키워드:", len(related_keywords), "개")
            df = pd.DataFrame({"인재상에 해당하는 키워드(중복제거)": related_keywords})
            print(df)

        except Exception as e:
            print("Error occurred:", e)
    else:
        print("검색 결과 URL을 가져오지 못했습니다.")